<a href="https://colab.research.google.com/github/cobusgreyling/LlamaIndex/blob/main/Step_wise_Controllable_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.0/136.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00


In [2]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 25.0 MB/s eta 0:00:00


In [3]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

In [6]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

tools = [multiply_tool, add_tool]

In [14]:
import os
import openai
os.environ['OPENAI_API_KEY'] = str("<Your API Key Goes Here>")

In [15]:
llm = OpenAI(model="gpt-3.5-turbo")

In [16]:
from llama_index.core.agent import AgentRunner
from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent

# Option 1: Initialize OpenAIAgent
agent = OpenAIAgent.from_tools(tools, llm=llm, verbose=True)


# # Option 2: Initialize AgentRunner with OpenAIAgentWorker
# openai_step_engine = OpenAIAgentWorker.from_tools(tools, llm=llm, verbose=True)
# agent = AgentRunner(openai_step_engine)

In [17]:
agent.chat("Hi")

Added user message to memory: Hi


AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[])

In [18]:
response = agent.chat("What is (121 * 3) + 42?")

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

=== Calling Function ===
Calling function: add with args: {"a": 363, "b": 42}
Got output: 405



In [19]:
response

AgentChatResponse(response='The result of (121 * 3) + 42 is 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [20]:
# start task
task = agent.create_task("What is (121 * 3) + 42?")

In [21]:
step_output = agent.run_step(task.task_id)

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {"a":121,"b":3}
Got output: 363



In [22]:
step_output

TaskStepOutput(output=AgentChatResponse(response='None', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363)], source_nodes=[]), task_step=TaskStep(task_id='4c984a54-0d9c-4d73-b070-ae8f2743b181', step_id='3d6e67fd-af9d-421b-80b0-f74baab09042', input='What is (121 * 3) + 42?', step_state={}, next_steps={}, prev_steps={}, is_ready=True), next_steps=[TaskStep(task_id='4c984a54-0d9c-4d73-b070-ae8f2743b181', step_id='23adbab6-0074-47fc-82d3-21c466bd857f', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)], is_last=False)

In [23]:
step_output = agent.run_step(task.task_id)

=== Calling Function ===
Calling function: add with args: {"a":363,"b":42}
Got output: 405



In [24]:
step_output = agent.run_step(task.task_id)

In [25]:
# display final response
print(step_output.is_last)

True


In [26]:
# now that the step execution is done, we can finalize response
response = agent.finalize_response(task.task_id)
print(str(response))

The result of (121 * 3) + 42 is 405.


In [27]:
llm = OpenAI(model="gpt-4-1106-preview")

In [28]:
from llama_index.core.agent import AgentRunner, ReActAgentWorker, ReActAgent

In [29]:
# Option 1: Initialize OpenAIAgent
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

# # Option 2: Initialize AgentRunner with ReActAgentWorker
# react_step_engine = ReActAgentWorker.from_tools(tools, llm=llm, verbose=True)
# agent = AgentRunner(react_step_engine)

In [30]:
agent.chat("Hi")

Thought: The user has greeted me, and I should respond in kind.
Answer: Hello! How can I assist you today?


AgentChatResponse(response='Hello! How can I assist you today?', sources=[], source_nodes=[])

In [31]:
response = agent.chat("What is (121 * 3) + 42?")

Thought: The user has asked for a calculation. I need to use the multiply tool to find the product of 121 and 3 first.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363
Thought: Now that I have the product of 121 and 3, which is 363, I need to add 42 to it using the add tool.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: (121 * 3) + 42 is equal to 405.


In [32]:
response

AgentChatResponse(response='(121 * 3) + 42 is equal to 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [33]:
# start task
task = agent.create_task("What is (121 * 3) + 42?")

In [34]:
step_output = agent.run_step(task.task_id)

Thought: The user has asked for a calculation. I need to use the multiply tool to find the product of 121 and 3 first, and then use the add tool to add 42 to the result.
Action: multiply
Action Input: {'a': 121, 'b': 3}
Observation: 363


In [35]:
step_output.output

AgentChatResponse(response='Observation: 363', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363)], source_nodes=[])

In [36]:
step_output = agent.run_step(task.task_id)

Thought: I have the result of the multiplication (121 * 3 = 363). Now I need to use the add tool to add 42 to 363.
Action: add
Action Input: {'a': 363, 'b': 42}
Observation: 405


In [37]:
step_output.output

AgentChatResponse(response='Observation: 405', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [38]:
step_output = agent.run_step(task.task_id)

Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of (121 * 3) + 42 is 405.


In [39]:
step_output.output

AgentChatResponse(response='The result of (121 * 3) + 42 is 405.', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[])

In [40]:
tasks = agent.list_tasks()
print(len(tasks))

3


In [41]:
task_state = tasks[-1]
task_state.task.input

'What is (121 * 3) + 42?'

In [42]:
# get completed steps
completed_steps = agent.get_completed_steps(task_state.task.task_id)

In [43]:
completed_steps[0]

TaskStepOutput(output=AgentChatResponse(response='Observation: 363', sources=[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)], source_nodes=[]), task_step=TaskStep(task_id='8d239a0b-2f88-4eb0-a4df-f523d1775559', step_id='e6516171-a065-460d-bb79-773e1fff445e', input='What is (121 * 3) + 42?', step_state={'is_first': False}, next_steps={}, prev_steps={}, is_ready=True), next_steps=[TaskStep(task_id='8d239a0b-2f88-4eb0-a4df-f523d1775559', step_id='419f1477-c29e-4988-923b-cc1ae71159b1', input=None, step_state={'is_first': False}, next_steps={}, prev_steps={}, is_ready=True)], is_last=False)

In [44]:
for idx in range(len(completed_steps)):
    print(f"Step {idx}")
    print(f"Response: {completed_steps[idx].output.response}")
    print(f"Sources: {completed_steps[idx].output.sources}")

Step 0
Response: Observation: 363
Sources: [ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]
Step 1
Response: Observation: 405
Sources: [ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]
Step 2
Response: The result of (121 * 3) + 42 is 405.
Sources: [ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]
